### Computing PD using bond prices

In [35]:
import time
from datetime import date
from datetime import datetime as dt
from datetime import timedelta

import numpy as np
import pandas as pd
import yfinance as yf
import sympy as sp # for solving equations
import pandas_datareader as dr

In [2]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [3]:
options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()), options=options
)

In [4]:
driver.get("http://finra-markets.morningstar.com/BondCenter/Results.jsp")

#### Define company name, company ticker and coupon frequency

In [5]:
company_ticker = "HES"
company_name = "Hess"
coupon_frequency = "Semi-Annual"

In [6]:
# click agree
WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, ".button_blue.agree"))
).click()

In [7]:
# click edit search
WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, "a.qs-ui-btn.blue"))
).click()

In [8]:
# input Issuer Name
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, "input[id=firscreener-issuer]"))
)
inputElement = driver.find_element(By.ID, "firscreener-issuer")
inputElement.send_keys(company_name)

In [9]:
# input Symbol
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, "input[id=firscreener-cusip]"))
)
inputElement = driver.find_element(By.ID, "firscreener-cusip")
inputElement.send_keys(company_ticker)

In [10]:
# click advanced search
WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, "a.ms-display-switcher.hide"))
).click()

In [11]:
# input Coupon Frequency
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, "select[name=interestFrequency]"))
)
Select(
    (driver.find_elements(By.CSS_SELECTOR, "select[name=interestFrequency]"))[0]
).select_by_visible_text(coupon_frequency)

In [12]:
# click show results
WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, "input.button_blue[type=submit]"))
).click()

In [13]:
# wait for results
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located(
        (By.CSS_SELECTOR, ".rtq-grid-row.rtq-grid-rzrow .rtq-grid-cell-ctn")
    )
)

<selenium.webdriver.remote.webelement.WebElement (session="d32107d85b567f1ed7216281ef8babef", element="3f33ab8a-6bfb-42de-a9ec-37e609f4f09f")>

In [14]:
# create DataFrame from scrape
frames = []
for page in range(1, 11):
    bonds = []
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located(
            (By.CSS_SELECTOR, (f"a.qs-pageutil-btn[value='{str(page)}']"))
        )
    )  # wait for page marker to be on expected page
    time.sleep(2)

    headers = [
        title.text
        for title in driver.find_elements(By.CSS_SELECTOR,
            ".rtq-grid-row.rtq-grid-rzrow .rtq-grid-cell-ctn"
        )[1:]
    ]

    tablerows = driver.find_elements(By.CSS_SELECTOR,
        "div.rtq-grid-bd > div.rtq-grid-row"
    )
    for tablerow in tablerows:
        tablerowdata = tablerow.find_elements(By.CSS_SELECTOR, "div.rtq-grid-cell")
        bond = [item.text for item in tablerowdata[1:]]
        bonds.append(bond)

        # Convert to DataFrame
        df = pd.DataFrame(bonds, columns=headers)

    frames.append(df)

    try:
        driver.find_element(By.CSS_SELECTOR, "a.qs-pageutil-next").click()
    except:  # noqa E722
        break

bond_prices_df = pd.concat(frames)

In [15]:
bond_prices_df

,Issuer Name,Symbol,Callable,Sub-Product Type,Coupon,Maturity,Moody's®,S&P,Price,Yield
0,HESS CORP,HES.GH,Yes,Corporate Bond,6.000,01/15/2040,Baa3,BBB-,100.389,5.963
1,HESS CORP,HES.GI,Yes,Corporate Bond,5.600,02/15/2041,Baa3,BBB-,95.175,6.041
2,HESS CORP,HES4136877,Yes,Corporate Bond,3.500,07/15/2024,Baa3,BBB-,97.309,5.337
3,HESS CORP,HES4405829,Yes,Corporate Bond,4.300,04/01/2027,Baa3,BBB-,95.564,5.480
4,HESS CORP,HES4405830,Yes,Corporate Bond,5.800,04/01/2047,Baa3,BBB-,96.549,6.073
5,HESS MIDSTREAM OPERATIONS LP,HES5392919,Yes,Corporate Bond,5.500,10/15/2030,Ba2,BB+,91.944,6.849
6,HESS MIDSTREAM OPERATIONS LP,HES4567499,Yes,Corporate Bond,5.625,02/15/2026,WR,BB+,104.500,4.431
7,HESS MIDSTREAM OPERATIONS LP,HES4927355,Yes,Corporate Bond,5.625,02/15/2026,,BB+,98.183,6.271
8,HESS MIDSTREAM OPERATIONS LP,HES5233164,Yes,Corporate Bond,4.250,02/15/2030,Ba2,BB+,86.000,6.756
9,HESS MIDSTREAM PARTNERS LP,HES4918686,Yes,Corporate Bond,5.125,06/15/2028,Ba2,BB+,93.530,6.553


In [19]:
def bond_dataframe_filter(df):
    df['Yield'].replace("", np.nan, inplace=True)
    df["Moody's®"].replace({"WR": np.nan, "": np.nan}, inplace=True)
    df["S&P"].replace({"NR": np.nan, "": np.nan}, inplace=True)
    df = df.dropna(subset=["Yield"])
    df = df.dropna(subset=["Moody's®"])
    df = df.dropna(subset=["S&P"])
    
    df["Yield"] = df["Yield"].astype(float)
    df["Coupon"] = df["Coupon"].astype(float)
    df["Price"] = df["Price"].astype(float)
    
    now = dt.strptime(date.today().strftime("%m/%d/%Y"), "%m/%d/%Y")
    df["Maturity"] = pd.to_datetime(df["Maturity"]).dt.strftime("%m/%d/%Y")
    
    daystillmaturity = []
    yearstillmaturity = []
    for maturity in df["Maturity"]:
        daystillmaturity.append((dt.strptime(maturity, "%m/%d/%Y") - now).days)
        yearstillmaturity.append((dt.strptime(maturity, "%m/%d/%Y") - now).days / 360)
    
    df["Maturity"] = pd.Series(daystillmaturity)
    df["Maturity Years"] = (round(pd.Series(yearstillmaturity) / 0.5) * 0.5)  # Better for Semi-Annual Payments
    df["Maturity"] = df["Maturity"].astype(float)
    years_mask = (df["Maturity Years"] > 0) & (df["Maturity Years"] <= 5)
    
    df = df.loc[years_mask]

    return df


In [20]:
bond_df_result = bond_dataframe_filter(bond_prices_df)
bond_df_result

,Issuer Name,Symbol,Callable,Sub-Product Type,Coupon,Maturity,Moody's®,S&P,Price,Yield,Maturity Years
2,HESS CORP,HES4136877,Yes,Corporate Bond,3.5,566.0,Baa3,BBB-,97.309,5.337,1.5
3,HESS CORP,HES4405829,Yes,Corporate Bond,4.3,1556.0,Baa3,BBB-,95.564,5.480,4.5


## Risk adjusted discount rate

\begin{equation*}
BOND\ PRICE = \frac{ECF_1}{1+d}\ +\ \frac{ECF_2}{(1+d)^2}\ +\ \frac{ECF_3}{(1+d)^3}
\end{equation*}

This discount rate includes risk free rate plus the risk premium associated with any bond (country + default risk premium).

$ECF_1 = (P)*(Default\ Payout)\ +\ (1-P)*(Coupon\ Payment)$<br>
$ECF_2 = (1-P)*\{(P)\ *(Default\ Payout)\ +\ (1-P)*(Coupon\ Payment)\}$<br>
$ECF_3 = (1-P)^2*\{(P)\ *(Default\ Payout)\ +\ (1-P)*(Coupon\ Payment\ +\ Principal)\}$<br>

$P = Probability\ of\ Default$<br>
$Default\ Payout = Principal\ *\ Recovery\ Rate$<br>

If we know the discounted rate, we can use to identify market implied default probability of any corporate bond which in turn will help up to estimate risk in investing in any company bond.

**_Risk-adjusted Discount Rate = Risk-free Interest Rate + Expected Risk Premium_**

**_Expected Risk Premium = (Market Rate of Return - Risk-free Rate of Return) * Beta_**


$$r_m = r_f\ +(\beta*MRP)$$<br>
$r_m = Market\ Rate\ of\ Return$<br>
$r_f = Riskfree\ Rate$<br>
$\beta = Beta$<br>
$MRP = Market\ Risk\ Premium$<br>

Beta can be approximated by selecting any proxy short term corporate bond

In [6]:
# Ten-Year Risk-free Rate

timespan = 100
current_date = date.today()
past_date = current_date - timedelta(days=timespan)

In [10]:
ten_year_risk_free_rate_df = yf.download("^TNX", past_date, current_date)

[*********************100%***********************]  1 of 1 completed


In [18]:
ten_year_risk_free_rate = (
    ten_year_risk_free_rate_df.iloc[len(ten_year_risk_free_rate_df) - 1, 3]
) / 100
ten_year_risk_free_rate

0.03886999845504761

In [22]:
""" From ashwath damodran published country wise risk premium list, we can get"""
# Market Risk Premium
market_risk_premium = 0.0472
# Market Equity Beta (its one only as we are comparing with itself)
stock_market_beta = 1

In [24]:
market_rate_of_return = ten_year_risk_free_rate + (stock_market_beta * market_risk_premium)
market_rate_of_return

0.0860699984550476

In [25]:
# One-Year Risk-free Rate
one_year_risk_free_rate = (1 + ten_year_risk_free_rate) ** (1 / 10) - 1
one_year_risk_free_rate

0.0038206383508816444

In [26]:
bond_fund_ticker = "VCSH"

In [27]:
market_data = yf.download("SPY", past_date, current_date)  # the market
fund_data = yf.download("VCSH", past_date, current_date)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


### Calculate beta using 2 approaches

In [28]:
fund_market_cov = fund_data["Adj Close"].cov(market_data["Adj Close"])
print("covariance between fund and market: ", fund_market_cov)

market_var = market_data["Adj Close"].var()
print("market variance: ", market_var)

bond_fund_beta_cv = fund_market_cov / market_var
print("bond fund beta (using covariance/variance): ", bond_fund_beta_cv)


covariance between fund and market:  8.519984819956546
market variance:  195.2614218405008
bond fund beta (using covariance/variance):  0.0436337333798383


In [31]:
bond_beta = bond_fund_beta_cv
bond_beta

0.0436337333798383

In [32]:
# Expected Risk Premium
expected_risk_premium = (market_rate_of_return - one_year_risk_free_rate) * bond_beta
expected_risk_premium

0.003588846649447487

In [33]:
risk_adjusted_discount_rate = one_year_risk_free_rate + expected_risk_premium
risk_adjusted_discount_rate

0.007409485000329131